In [ ]:
instance = instanceId = InstanceId
apiUrl = ApiUrl
schoolYear = SchoolYear
DistrictId = DistrictID = districtId = districtID
apiLimit = batchLimit

prepareEdFiMetaData = prepareEdFiMetadata

### URL Initializations

In [22]:
%run OEA/modules/Ed-Fi/v0.6/src/utilities/edfi_v0_6_fetch_urls

In [23]:
instance_id = instanceId
school_year = schoolYear
api_url = apiUrl

edfi_api_manager = EdFiApiManager(api_url, instance_id, school_year)
edfi_api_manager.update_urls()
edfi_api_manager.set_other_metadata()

dependenciesUrl = edfi_api_manager.dependencies_url
openApiMetadataUrl = edfi_api_manager.openapi_metadata_url
dataManagementUrl = edfi_api_manager.data_management_url
authUrl = edfi_api_manager.auth_url

changeQueriesUrl = edfi_api_manager.get_referenced_url('Change-Queries')
changeQueriesUrl = changeQueriesUrl[:-13].replace('/metadata/', '/')
swagger_url = swaggerUrl = edfi_api_manager.get_referenced_url('Resources')

apiVersion = edfi_api_manager.api_version
apiVersion = apiVersion[1:] if apiVersion.startswith('v') else apiVersion

### OEA Initializations

In [24]:
%run OEA/modules/Ed-Fi/v0.6/src/utilities/edfi_v0_6_edfi_py

In [25]:
oea = EdFiOEAChild()   
oea.set_workspace(workspace)

In [26]:
parameterized = False
if parameterized == True:
    edfiEntitiesPath = f'stage1/Transactional/{moduleName}/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/etl_entities/current_run_data'

    _, edfiEntities = edfi.listSpecifiedEntities(edfiEntitiesPath)
else:
    edfiEntities = "All"  

### Main Code

In [33]:
def ingest_edfi_dataset(tables_source, 
                        items = 'All'):
    full_source_path = f"stage1/Transactional/{tables_source}"
    if items == 'All':
        items = oea.get_folders(full_source_path) 
   
    options = {'source_format': 'json', 'multiline': False}
    #items = ['assessmentItems']
    for item in items: 
        table_path = full_source_path +'/'+ item
        try:
            entity_path = f"{tables_source}/{item}"
            if item == 'metadata.csv' or item == 'parameterizedRunLogs':
                logger.info('ignore metadata processing, since this is not a table to be ingested')
            else:
                oea.ingest(entity_path, 'id', options)
        
        except AnalysisException as e:
            # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
            pass
        except Exception as error:
            # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
            logger.error(error)

In [ ]:
from datetime import datetime
import math
source_path = f'stage1/Transactional/Ed-Fi/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/metadata-assets/frequency_etl.csv'  
destination_path = source_path #f'stage1/Transactional/Ed-Fi/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/metadata-assets/frequency_based_etl.csv'  
logs_path = f"stage1/Transactional/Ed-Fi/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/metadata-assets/_frequency_etl_logs/run_logs_{datetime.today().strftime('%Y-%m-%d')}.csv"

processor = EntityFrequencyProcessor(oea = oea, 
                                     filepath = source_path, 
                                     highFrequentDelta = highFrequentDelta,#0.005, 
                                     moderateFrequentDelta = moderateFrequentDelta, #5, 
                                     lowFrequentDelta = lowFrequentDelta, #10, 
                                     descriptorsDelta = descriptorsDelta) #360)

processor.load_lookup_df()
_, entities_to_etl = processor.return_entities_to_etl()

logger.info(entities_to_etl)

edfiEntities = "All" #['schoolYearTypes']
tpdmEntities = 'All'

edfiEntities = entities_to_etl.get('ed-fi', [])
tpdmEntities = entities_to_etl.get('tpdm', [])

In [34]:
from datetime import datetime
ingest_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/ed-fi',
                    edfiEntities)
ingest_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/tpdm',
                    tpdmEntities)